<a href="https://colab.research.google.com/github/MMathisLab/DeepLabCut/blob/master/examples/COLAB/COLAB_transformer_reID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut 2.2 Toolbox Demo on how to use our Pose Transformer for unsupervised identity tracking of animals
![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1628250004229-KVYD7JJVHYEFDJ32L9VJ/DLClogo2021.jpg?format=1000w)

https://github.com/DeepLabCut/DeepLabCut

### This notebook illustrates how to use the transformer for a multi-animal DeepLabCut (maDLC) Demo 3 mouse project:
- load our mini-demo data that includes a pretrained model and unlabeled video.
- analyze a novel video.
- use the transformer to do unsupervised ID tracking.
- create quality check plots and video.

### To create a full maDLC pipeline please see our full docs: https://deeplabcut.github.io/DeepLabCut/docs/intro.html 
- Of interest is a full how-to for maDLC: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html
- a quick guide to maDLC: https://deeplabcut.github.io/DeepLabCut/docs/tutorial.html
- a demo COLAB for how to use maDLC on your own data: https://github.com/DeepLabCut/DeepLabCut/blob/master/examples/COLAB_maDLC_TrainNetwork_VideoAnalysis.ipynb

### To get started, please go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [1]:
# Install the latest DeepLabCut version:
!pip install https://github.com/DeepLabCut/DeepLabCut/archive/master.zip
#!pip install git+https://github.com/DeepLabCut/DeepLabCut.git@mwm/doploymentTransformer

     - 75.0 MB 745 kB/s
     |████████████████████████████████| 177 kB 12.4 MB/s 
     |████████████████████████████████| 109 kB 42.0 MB/s 
     |████████████████████████████████| 948 kB 50.3 MB/s 
     |████████████████████████████████| 3.3 MB 69.0 MB/s 
     |████████████████████████████████| 11.2 MB 74.6 MB/s 
     |████████████████████████████████| 29.2 MB 1.3 MB/s 
     |████████████████████████████████| 9.8 MB 25.5 MB/s 
     |████████████████████████████████| 296 kB 76.1 MB/s 
     |████████████████████████████████| 352 kB 62.1 MB/s 
     |████████████████████████████████| 900 kB 58.0 MB/s 
     |████████████████████████████████| 34.5 MB 14 kB/s 
     |████████████████████████████████| 546 kB 54.7 MB/s 
     |████████████████████████████████| 462 kB 73.9 MB/s 
  Created wheel for deeplabcut: filename=deeplabcut-2.2.1rc1-py3-none-any.whl size=588743 sha256=2d526459557551371f61a9dbda923489dcdee97be265934cad801a85e954f6a4
  Stored in directory: /tmp/pip-ephem-wheel-cache-3fj0iaxk/w

No information needs edited in the cells below, you can simply click run on each:

### Download our Demo Project from our server:

In [1]:
# Download our demo project:
import urllib.request
from io import BytesIO
from zipfile import ZipFile

def unzip_from_url(url, dest_folder=''):
    # Directly extract files without writing the archive to disk
    resp = urllib.request.urlopen(url)
    with ZipFile(BytesIO(resp.read())) as zf:
        zf.extractall(path=dest_folder)


project_url = "http://deeplabcut.rowland.harvard.edu/datasets/demo-me-2021-07-14.zip"
unzip_from_url(project_url, "/content")

## Analyze a novel 3 mouse video with our maDLC DLCRNet, pretrained on 3 mice data 

###in one step, since auto_track=True you extract detections and association costs, create tracklets, & stitch them. We can use this to compare to the transformer-guided tracking below.


In [2]:
import deeplabcut as dlc
import os

project_path = "/content/demo-me-2021-07-14"
config_path = os.path.join(project_path, "config.yaml")
video = os.path.join(project_path, "videos", "videocompressed1.mp4")

dlc.analyze_videos(config_path,[video], shuffle=0, videotype="mp4",auto_track=True)

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)
Using snapshot-20000 for model /content/demo-me-2021-07-14/dlc-models/iteration-0/demoJul14-trainset95shuffle0


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:684: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  outputs = layer.apply(inputs, training=is_training)


Activating extracting of PAFs
Starting to analyze %  /content/demo-me-2021-07-14/videos/videocompressed1.mp4
Loading  /content/demo-me-2021-07-14/videos/videocompressed1.mp4
Duration of video [s]:  77.67 , recorded with  30.0 fps!
Overall # of frames:  2330  found with (before cropping) frame dimensions:  640 480
Starting to extract posture from the video(s) with batchsize: 8


100%|██████████| 2330/2330 [02:17<00:00, 16.99it/s]


Video Analyzed. Saving results in /content/demo-me-2021-07-14/videos...


...

100%|██████████| 2330/2330 [00:08<00:00, 279.58it/s]
2330it [00:07, 318.93it/s]


The tracklets were created (i.e., under the hood deeplabcut.convert_detections2tracklets was run). Now you can 'refine_tracklets' in the GUI, or run 'deeplabcut.stitch_tracklets'.
Processing...  /content/demo-me-2021-07-14/videos/videocompressed1.mp4


100%|██████████| 3/3 [00:00<00:00, 23172.95it/s]

The videos are analyzed. Time to assemble animals and track 'em... 
 Call 'create_video_with_all_detections' to check multi-animal detection quality before tracking.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


'DLC_dlcrnetms5_demoJul14shuffle0_20000'

### Next, you compute the local, spatio-temporal grouping and track body part assemblies frame-by-frame:

## Create a pretty video output:

In [3]:
#Filter the predictions to remove small jitter, if desired:
dlc.filterpredictions(config_path, 
                                 [video], 
                                 shuffle=0,
                                 videotype='mp4', 
                                 )

dlc.create_labeled_video(
    config_path,
    [video],
    videotype='mp4',
    shuffle=0,
    color_by="individual",
    keypoints_only=False,
    draw_skeleton=True,
    filtered=True,
)

Filtering with median model /content/demo-me-2021-07-14/videos/videocompressed1.mp4
Saving filtered csv poses!
Starting to process video: /content/demo-me-2021-07-14/videos/videocompressed1.mp4
Loading /content/demo-me-2021-07-14/videos/videocompressed1.mp4 and data.
Duration of video [s]: 77.67, recorded with 30.0 fps!
Overall # of frames: 2330 with cropped frame dimensions: 640 480
Generating frames and creating video.


100%|██████████| 2330/2330 [00:40<00:00, 57.85it/s]


Now, on the left panel if you click the folder icon, you will see the project folder "demo-me.."; click on this and go into "videos" and you can find the "..._id_labeled.mp4" video, which you can double-click on to download and inspect!

### Create Plots of your data:

> after running, you can look in "videos", "plot-poses" to check out the trajectories! (sometimes you need to click the folder refresh icon to see it). Within the folder, for example, see plotmus1.png to vide the bodyparts over time vs. pixel position.



In [ ]:
dlc.plot_trajectories(config_path, [video], shuffle=0,videotype='mp4')

# Transformer for reID

while the tracking here is very good without using the transformer, we want to demo the workflow for you! 

In [4]:
dlc.transformer_reID(config_path, [video], 
                     shuffle=0, videotype='mp4', 
                     track_method='ellipse',n_triplets=100
                     )

Using snapshot-20000 for model /content/demo-me-2021-07-14/dlc-models/iteration-0/demoJul14-trainset95shuffle0


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:684: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  outputs = layer.apply(inputs, training=is_training)


Activating extracting of PAFs
Starting to analyze %  /content/demo-me-2021-07-14/videos/videocompressed1.mp4
Loading  /content/demo-me-2021-07-14/videos/videocompressed1.mp4
Duration of video [s]:  77.67 , recorded with  30.0 fps!
Overall # of frames:  2330  found with (before cropping) frame dimensions:  640 480
Starting to extract posture


100%|██████████| 2330/2330 [00:21<00:00, 110.12it/s]
/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_tracking_pytorch/tracking_utils/preprocessing.py:54: RuntimeWarning: Mean of empty slice
  match_id = np.argmin(np.nanmean(diff, axis=(1, 2)))


If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
Epoch 10, train acc: 0.61
Epoch 10, test acc 0.35
Epoch 20, train acc: 0.73
Epoch 20, test acc 0.35
Epoch 30, train acc: 0.75
Epoch 30, test acc 0.35
Epoch 40, train acc: 0.75
Epoch 40, test acc 0.35
Epoch 50, train acc: 0.74
Epoch 50, test acc 0.30
Epoch 60, train acc: 0.81
Epoch 60, test acc 0.30
Epoch 70, train acc: 0.78
Epoch 70, test acc 0.40
Epoch 80, train acc: 0.76
Epoch 80, test acc 0.35
Epoch 90, train acc: 0.76
Epoch 90, test acc 0.40
Epoch 100, train acc: 0.82
Epoch 100, test acc 0.35
loading params
Processing...  /content/demo-me-2021-07-14/videos/videocompressed1.mp4


100%|██████████| 3/3 [00:00<00:00, 26269.13it/s]


now we can make another video with the transformer-guided tracking:


In [5]:
dlc.plot_trajectories(config_path, [video], 
                      shuffle=0,videotype='mp4', 
                      track_method="transformer"
                      )

Loading  /content/demo-me-2021-07-14/videos/videocompressed1.mp4 and data.
Plots created! Please check the directory "plot-poses" within the video directory


In [7]:
dlc.create_labeled_video(
    config_path,
    [video],
    videotype='mp4',
    shuffle=0,
    color_by="individual",
    keypoints_only=False,
    draw_skeleton=True,
    track_method="transformer"
)

Starting to process video: /content/demo-me-2021-07-14/videos/videocompressed1.mp4
Loading /content/demo-me-2021-07-14/videos/videocompressed1.mp4 and data.
Duration of video [s]: 77.67, recorded with 30.0 fps!
Overall # of frames: 2330 with cropped frame dimensions: 640 480
Generating frames and creating video.


100%|██████████| 2330/2330 [00:40<00:00, 57.23it/s]
